In [1]:
import sys
sys.path.insert(0, '..')
from app.SIWeaviateClient import SIWeaviateClient
from app.processing.image.SIImageDescription import SIImageDescription
from app.processing.text.SIITranslator import SITranslator

image_descriptor = SIImageDescription()
translator = SITranslator()



/home/erwan/anaconda3/envs/scienceinfuse/lib/python3.9/site-packages/timm/models/layers/__init__.py:48: DeprecationWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", DeprecationWarning)


In [ ]:
from app.processing.PDFProcessor import PDFProcessor
import os
import glob

def get_all_pdfs():
    pdf_files = []
    for root, dirs, files in os.walk("./data"):
        for file in files:
            if file.endswith(".pdf"):
                pdf_files.append(os.path.join(root, file))
    return pdf_files


with SIWeaviateClient() as client:
    pdf_paths = get_all_pdfs()
    for pdf_path in pdf_paths[:1]:
        print("==========================================")
        print(pdf_path)
        pdf = PDFProcessor(client, image_descriptor, translator, pdf_path)

